In [1]:
import os

In [7]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\Spam-email\\research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\Spam-email'

In [24]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    x_test_path: Path
    x_train_path: Path
    y_test_path: Path
    y_train_path: Path
    model_name: str
    loss: str
    metrics: list
    target_column: str

In [25]:
from Spam_mail.utils.common import *
from Spam_mail.constants import *

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Simple_nn
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            x_test_path= config.x_test_path,
            x_train_path= config.x_train_path,
            y_test_path= config.y_test_path,
            y_train_path= config.y_train_path,
            model_name= config.model_name,
            loss= params.loss,
            metrics= params.metrics,
            target_column= schema.Category
        )

        return model_trainer_config

In [27]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model

In [28]:
class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train_model(self, x_train, x_test, y_train, y_test):
        logger.info('Building Model')
        # Build the Deep Learning Model
        model = Sequential()
        model.add(Dense(64, activation='relu', input_dim=x_train.shape[1]))
        model.add(Dropout(0.7))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        model.summary()
        logger.info('Model Built')
        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        # Train the model
        history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping], verbose=1)
        logger.info('Model Trained')
        
        # Save the trained model
        save_path = os.path.join(self.config.root_dir, self.config.model_name)
        save_model(model, save_path)

        logger.info(' Model Saved')
        # Evaluate the model
        y_pred_prob = model.predict(x_test)
        y_pred = (y_pred_prob > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, pos_label=1)
        recall = recall_score(y_test, y_pred, pos_label=1)
        f1 = f1_score(y_test, y_pred, pos_label=1)

        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1 score: {f1:.4f}')

        # Visualize training history
        plt.figure(figsize=(10, 6))

        # Plot training & validation accuracy values
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')

        # Plot training & validation loss values
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')

        plt.tight_layout()
        plt.show()

In [29]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train_model()
except Exception as e:
    raise e

[2024-07-04 11:21:58,093: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-04 11:21:58,093: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-04 11:21:58,093: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-04 11:21:58,093: INFO: common: created directory at: artifacts]
[2024-07-04 11:21:58,093: INFO: common: created directory at: artifacts/model_trainer]


TypeError: ModelTrainer.train_model() missing 4 required positional arguments: 'x_train', 'x_test', 'y_train', and 'y_test'